<a href="https://colab.research.google.com/github/peterrrock2/gerrychain_migration_settings/blob/main/GerryChain_Migration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash

version="old"
# version="new"

if [ ! -f gerrymandria.json ]
then
  wget -q https://raw.githubusercontent.com/mggg/GerryChain/refs/heads/main/docs/_static/gerrymandria.json
  wget -q https://raw.githubusercontent.com/mggg/GerryChain/refs/heads/main/docs/_static/05_bg_census_consolidated.json -O BG_05.json
  wget -q https://github.com/mggg-states/PA-shapefiles/raw/refs/heads/master/PA.zip
  mkdir PA; unzip -qq PA.zip -d PA

  if [[ $version == "old" ]]
  then
    wget -q https://raw.githubusercontent.com/peterrrock2/gerrychain_migration_settings/refs/heads/main/old_gc/pyproject.toml
    wget -q https://raw.githubusercontent.com/peterrrock2/gerrychain_migration_settings/refs/heads/main/old_gc/uv.lock
  else
    wget -q https://raw.githubusercontent.com/peterrrock2/gerrychain_migration_settings/refs/heads/main/new_gc/pyproject.toml
    wget -q https://raw.githubusercontent.com/peterrrock2/gerrychain_migration_settings/refs/heads/main/new_gc/uv.lock

    echo "Please restart the runtime"
  fi

  # Point uv's "project environment" at the running Python's prefix
  export UV_PROJECT_ENVIRONMENT="$(python -c 'import sysconfig; print(sysconfig.get_config_var("prefix"))')"

  # Install exactly what's in uv.lock WITHOUT rewriting it
  # --inexact avoids removing Colab's preinstalled packages
  # --no-install-project installs deps but not local project as a package
  uv sync -q --frozen --inexact --no-dev --no-install-project
fi

#Task 1

Set up a basic ReCom chain on `gerrymandria.json` with

- A population and cut-edge updater
- Assigment using "district"
- Ideal population derived from the population updater
- Allowable popluation deviation of 1%
- 5000 steps

In [ ]:
from gerrychain import Graph, Partition, MarkovChain, Election, GeographicPartition
from gerrychain.updaters import cut_edges, Tally
from gerrychain.accept import always_accept
from gerrychain.tree import bipartition_tree
from gerrychain.proposals import recom
from gerrychain.optimization import SingleMetricOptimizer


import pandas as pd
import numpy as np
from functools import partial


In [ ]:
graph = # Your Code Here

my_updaters = {
  # Your Code Here
}

initial_partition = Partition(
  # Your Code Here
)

ideal_population =  # Your Code Here

proposal = partial(
  # Your Code Here
)

recom_chain = MarkovChain(
  # Your Code Here
)

In [ ]:
assignment_list = []

for item in recom_chain.with_progress_bar():
    assignment_list.append(item.assignment)

#Task 2

Set up a region-aware ReCom chain on `gerrymandria.json` with the same settings as before

- A population derived from the "TOTPOP" column and cut-edge updater
- Assigment using "district"
- Ideal population derived from the population updater
- Allowable popluation deviation of 1%
- 5000 steps

but with the following modifications:

- Surcharge "muni" by 0.5
- Surcharge "water_dist" by 0.5
- Change the bipartition_tree method to use 'random.choice' for selecting a cut edge.

In [ ]:
# Your Code Here

In [ ]:
assignment_list = []

for item in recom_chain.with_progress_bar():
    assignment_list.append(item.assignment)

#Task 3

Set up a ReCom chain on "PA_VTDs.json"

- A population updater derived from the "TOTPOP" column.
- Election updaters for the "PRES12" and "PRES16" elections
- Initial Partition as a `GeographicPartition`
- Assignment using "538DEM"
- Ideal population derived from the population updater
- Allowable popluation deviation of 2%
- 200 steps



In [ ]:
graph = # Your Code Here

In [ ]:
elections = [
  # Your Code Here
]

my_updaters = {"population": Tally("TOTPOP", alias="population")}
election_updaters = {election.name: election for election in elections}
my_updaters.update(election_updaters)

initial_partition = GeographicPartition(
  # Your Code Here
)

ideal_population = # Your Code Here

proposal = partial(
  # Your Code Here
)

chain = MarkovChain(
    proposal=proposal,
    constraints=[],
    accept=always_accept,
    initial_state=initial_partition,
    total_steps=200
)


In [ ]:
data = pd.DataFrame(
    sorted(partition["SEN12"].percents("Democratic"))
    for partition in chain.with_progress_bar()
)

#Task 5

Set up a short-bursts SingleMetricOptimizer

- A population updater derived from the "tot_pop_20" column.
- Popluation updaters for the "VAP" and "BVAP" columns
- Assignment from a random assignment with 35 districts and population deviation of 2%
- Ideal population derived from the population updater
- Allowable popluation deviation of 2%
- 200 bursts of length 5
- Minimizes the number of cut edges


In [ ]:
graph = # Your Code Here

POPCOL = # Your Code Here
N_DISTS = # Your Code Here
EPS = # Your Code Here
TOTPOP = sum(graph.nodes()[n][POPCOL] for n in graph.nodes())

chain_updaters = {
  # Your Code Here
}

initial_partition = Partition.from_random_assignment(
  # Your Code Here
)

proposal = partial(
    recom,
    pop_col=POPCOL,
    pop_target=TOTPOP/SEN_DISTS,
    epsilon=EPS,
)


def num_cut_edges(partition):
    # Your Code Here
    pass

optimizer = SingleMetricOptimizer(
  # Your Code Here
)


In [ ]:
total_steps = 1000
burst_length = 5

min_scores_sb = np.zeros(total_steps)
for i, part in enumerate(optimizer.short_bursts(burst_length, total_steps//burst_length, with_progress_bar=True)):
    min_scores_sb[i] = optimizer.best_score